In [1]:
#from pip._internal import main as pip_main
#except ImportError:
#    from pip._internal import main as pip_main

#package_to_install = "numpy"
#try:
#    pip_main(["install", "--user", package_to_install])
#except SystemExit as e:
#    print(e)

The Newtonian Gravitational Force depends upon the masses of the two objects and the square of the distances between them. When one astronomical body acts upon another with force F12, the second body acts upon the first with equal and opposite force F21=-F12. Gravitational Force follows an inverse square law with regard to distance scaling that is exact in Newtonian gravity.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

def NewtonianForce(mass1,mass2, r):
    return (6.67408*10**-11)*mass1*mass2/r/r;

To compute an orbit, it is necessary to solve a second order differential equation. 

F=ma

For a two body problem:

F12x=m1 d^2 x1/dt^2

This can be decomposed into two coupled differential equations.

F12x= m1 d v1/dt

v1= d x1/dt

There is also a y component and a z component. 
It is also necessary to track the position of the second particle.

Recall that once a force F12 is split into its components in the x and y direction, F12 will no longer exactly be given by the NewtonainForce equation given above but rather the magnitude of F12x, F12y, and F12z combined will equal that total. The nagnitude of a vector is found by squaring and summing the components then taking the square roots. So, sqrt(F12x^2+F12y^2+F12z^2)=FNewtonianForce

Assume, for the purposes of a two body problem, the orbit is confined to the x-y plane. 

Define polar coordinates phi in the x-y plane beginning from the x axis and proceeding toward the y axis. Then F12x=F12 cos phi and F12y= F12 sin phi.

F12x=x/sqrt(x^2+y^2)*F12

F12y = y/sqrt(x^2+y^2)*F12

# Numerical solutions to differential equations

To integrate a differential equation numerically, simply differentiate both of the two coupled equations, one time steop at a time, numerically. But how is this done numerically? There are a couple of ways. First consider how a derivative is defined. A derivative is a limit of a difference divided by a timestep. The euler derivative makes this definition discrete for small time steps. 

In [22]:
def euler(h,t, x,y,z,f,debugprint):
    xstep = (f(t, x, y,z)+f(t+h,x+h,y,z))/2*h
    xnew=x+xstep
    if(debugprint):
        print(h,t,x,xstep,xnew)
    return t+h,xnew

The euler routine essentially just finds the slope in a small region of the graph. 

The Runga Kutta numerical derivative is a fourth order approximation to the derivative that accounts for the curvature and is better for something like an orbit that fundamentally has curvature to it where orvershoot near a minima may matter. 

# Verify Euler numerically

In [23]:
def polynomial(x,power):
    p=x**power
    #print(pi)
    return p
    

def polymaker(t,x,y,z,power):
    def poly(t,x,y,z):
        return polynomial(x,power)
    return poly

#exact solution to pow=1 is lnx

In [24]:
import numpy as np
t=0.
x=1.
y=1.
z=1.
print(t,x)
polyfn=polymaker(t,x,y,z,2)

0.0 1.0


In [25]:
outp=polyfn(t,x,y,z) #this is correct: x^2
print(outp)

1.0


In [26]:
eulerpolyout=euler(.01,t,x,y,z,polyfn,True)
print(eulerpolyout)

0.01 0.0 1.0 0.010100500000000002 1.0101005
(0.01, 1.0101005)


In [27]:
dt=0.01
numsteps=10
xvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    #t=0.+i*dt
    t,x=euler(dt,t,x,y,z,polyfn,True)
    print(x)
    xvals.append(x)
    tvals.append(t)
print(xvals)
print(tvals)

0.01 0.0 1.0 0.010100500000000002 1.0101005
1.0101005
0.01 0.01 1.0101005 0.010304540251002501 1.0204050402510025
1.0204050402510025
0.01 0.02 1.0204050402510025 0.0105148049657216 1.030919845216724
1.030919845216724
0.01 0.03 1.030919845216724 0.010731549257138418 1.0416513944738626
1.0416513944738626
0.01 0.04 1.0416513944738626 0.010955041415540812 1.0526064358894034
1.0526064358894034
0.01 0.05 1.0526064358894034 0.011185563732346866 1.0637919996217502
1.0637919996217502
0.01 0.060000000000000005 1.0637919996217502 0.011423413384554593 1.0752154130063047
1.0752154130063047
0.01 0.07 1.0752154130063047 0.011668903384963816 1.0868843163912685
1.0868843163912685
0.01 0.08 1.0868843163912685 0.011922363603812278 1.0988066799950809
1.0988066799950809
[1.0101005, 1.0204050402510025, 1.030919845216724, 1.0416513944738626, 1.0526064358894034, 1.0637919996217502, 1.0752154130063047, 1.0868843163912685, 1.0988066799950809]
[0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09

In [15]:
def analyticsoln1(array):
        returnarray=np.exp(array)
        return returnarray
print(analyticsoln1(tvals))

[1.01005017 1.02020134 1.03045453 1.04081077 1.0512711  1.06183655
 1.07250818 1.08328707 1.09417428]


# Implement RK4

In [46]:
def RK4(h,t,x,y,z,f): #not a finite difference so no step in y
    print(t,x,y,z,h)
    k1= h*f(t,x,y,z)
    print("k1",k1)
    k2=h*f(t+h/2.,x+k1/2.,y,z)
    print("k2",k2)
    k3=h*f(t+h/2.,x+k2/2.,y,z)
    print("k3",k3)
    k4=h*f(t+h,x+k3,y,z)
    print("k4",k4)
    return t+h, x+1/6.*(k1+2.*k2+2.*k3+k4)



In [5]:
def RK4nodebug(h,t,x,y,z,f): #not a finite difference so no step in y
    k1= h*f(t,x,y,z)
    #print("k1",k1)
    k2=h*f(t+h/2.,x+k1/2.,y,z)
    #print("k2",k2)
    k3=h*f(t+h/2.,x+k2/2.,y,z)
    #print("k3",k3)
    k4=h*f(t+h,x+k3,y,z)
    #print("k4",k4)
    return t+h, x+1/6.*(k1+2.*k2+2.*k3+k4)



# Verifying the RK4 routine numerically

First I verify the RK4 by examining the behavior for a differential equation where the right hand side is a fourth order polynomial. This should have a truncation error that is on the order of the fourth order polynomial itself

In [6]:
def polynomial4(t,x,a,b,c,d,e):
    p=a*t**4+b*t**3+c*t**2+d*t+e
    #print(pi)
    return p
    

def polymaker(t,x,y,z,a,b,c,d,e):
    def poly(t,x,y,z):
        return polynomial4(t,x,a,b,c,d,e)
    return poly
        
        

In [7]:
import numpy as np
t=0.
x=0.1
y=0.1
z=0.1
print(t,x)
polyfn=polymaker(t,x,y,z,1.,0.,0.,0.,0.)

0.0 0.1


In [8]:
#import matplotlib.pyplot as plt
#plt.plot(t,polyfn(t,x))

In [9]:
outp=polyfn(t,x,y,z) #this is correct to order of magnitude
print(outp)

0.0


In [10]:
rk4polyout=RK4(.01,t,x,y,z,polyfn)
print(x)
print(outp+x)
print(rk4polyout)

0.0 0.1 0.1 0.1 0.01
k1 0.0
k2 6.25e-12
k3 6.25e-12
k4 1e-10
0.1
0.1
(0.01, 0.10000000002083334)


In [11]:
dt=0.01
numsteps=2
intvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    t=0.+i*dt
    tnew,valueofintegral=RK4(dt,t,x,y,z,polyfn)
    #print(valueofintegral)
    intvals.append(valueofintegral)
    tvals.append(t)
print(intvals)
print(tvals)

0.01 0.1 0.1 0.1 0.01
k1 1e-10
k2 5.062499999999999e-10
k3 5.062499999999999e-10
k4 1.6e-09
[0.10000000062083333]
[0.01]


k1 1e-10
k2 5.062499999999999e-10
k3 5.062499999999999e-10
k4 1.6e-09
[6.208333333333332e-10]
[0.01]

In [12]:
2* 5.062499999999999e-10+ 2* 5.062499999999999e-10 

2.0249999999999995e-09

In [13]:
2.0249999999999995e-09+1.6e-09+1e-10

3.7249999999999994e-09

In [14]:
polyfn(0.1,0.1,0.1,0.1)

0.00010000000000000002

In [15]:
.01*polyfn(0.01,0.1,0.1,0.1)

1e-10

In [16]:
3.7249999999999994e-09 /6.

6.208333333333332e-10

# 1/6.*(k1+2.*k2+2.*k3+k4)

The value returned is not analyitically correct for the first step but I don't see the error yet.  

In [17]:
dt=0.01
numsteps=30
integralvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    t=0.+i*dt
    tnew,valueofintegral=RK4nodebug(dt,t,x,y,z,polyfn)
    #print(valueofintegral)
    integralvals.append(valueofintegral)
    tvals.append(t)
print(np.array(integralvals)-.1)
print(tvals)
print(len(integralvals),len(tvals))

[6.20833329e-10 4.22083334e-09 1.56208333e-08 4.20208333e-08
 9.30208333e-08 1.80620833e-07 3.19220833e-07 5.25620833e-07
 8.19020833e-07 1.22102083e-06 1.75562083e-06 2.44922083e-06
 3.33062083e-06 4.43102083e-06 5.78402083e-06 7.42562083e-06
 9.39422083e-06 1.17306208e-05 1.44780208e-05 1.76820208e-05
 2.13906208e-05 2.56542208e-05 3.05256208e-05 3.60600208e-05
 4.23150208e-05 4.93506208e-05 5.72292208e-05 6.60156208e-05
 7.57770208e-05]
[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29]
29 29


The analyitic solution to the differential equation

dx/dt = t^4 

is 

x= 1/5 * t^5

Below the output is shown. The results are plotted for scaling comparison. It doesn't look like it scales correctly. '

In [18]:
def ODEsolnanalytic(t,t0,x0):
    return 0.2*(t)**5.+x0

In [19]:
solnanalytic=[]
for i in np.arange(len(tvals)):
    solnanalytic.append(ODEsolnanalytic(tvals[i],tvals[0],0.1))
print(np.array(solnanalytic)-.1)
print(len(solnanalytic))

[2.00000017e-11 6.39999997e-10 4.86000000e-09 2.04800000e-08
 6.25000000e-08 1.55520000e-07 3.36140000e-07 6.55360000e-07
 1.18098000e-06 2.00000000e-06 3.22102000e-06 4.97664000e-06
 7.42586000e-06 1.07564800e-05 1.51875000e-05 2.09715200e-05
 2.83971400e-05 3.77913600e-05 4.95219800e-05 6.40000000e-05
 8.16820200e-05 1.03072640e-04 1.28726860e-04 1.59252480e-04
 1.95312500e-04 2.37627520e-04 2.86978140e-04 3.44207360e-04
 4.10222980e-04]
29


In [20]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(tvals,integralvals, size=15, line_color="blue", fill_color="blue", fill_alpha=0.5)
p.circle(tvals,solnanalytic, size=15, line_color="green",fill_color="green", fill_alpha=0.5)
#p.circle(tvals,np.array(solnanalytic)-np.array(integralvals),size=15, line_color="red", fill_color="red", fill_alpha=0.5)

show(p) # show the results

Loading BokehJS ...

The truncation error is on the order of the fifth order analytic solution, not the fourth order polynomial right hand side of the Ordinary Differential Equation.

In [21]:
print(rk4polyout) 
#There are multiple items in the y output, which is confusing. trace this over time and plot an evolution and compare to exact solution, which I have analytically written down. steve points out truncation error
print(((rk4polyout[1]-x-outp)/x))
#order4 polynomial error (comparable in size to polyfn itself, relative error with it is one)

(0.01, 0.10000000002083334)
2.0833335057091062e-10


# Try using euler funciton to compare

In [22]:
dt=0.01
numsteps=2
eintegralvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    t=0.+i*dt
    valueofintegral=euler(dt,t,x,y,z,polyfn,True)
    #print(valueofintegral)
    eintegralvals.append(valueofintegral)
    tvals.append(t)
print(np.array(eintegralvals))
print(tvals)

0.01 0.01 0.1
step1 1e-08
step2 1.6e-07
rise 1.5e-07
run 0.02
slope 7.499999999999999e-06
7.499999999999999e-06
[7.5e-06]
[0.01]


Find slope. First find rise. Then divide by run (2h). 

step2-step1 

f(t+h,x+h)-f(t,x)

In [23]:
1.6e-07-1e-08

1.5e-07

In [24]:
1.5e-07/2/0.01

7.499999999999999e-06

In [25]:
dt=0.01
numsteps=30
eintegralvals=[]
tvals=[]
for i in np.arange(1,numsteps):
    t=0.+i*dt
    valueofintegral=eulernodebug(dt,t,x,y,z,polyfn)
    #print(valueofintegral)
    eintegralvals.append(valueofintegral)
    tvals.append(t)
print(np.array(eintegralvals))
print(tvals)
print(len(eintegralvals),len(tvals))

NameError: name 'eulernodebug' is not defined

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(plot_width=400, plot_height=400, y_axis_type="log")

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(tvals,np.array(integralvals)-0.1, size=15, line_color="blue", fill_color="blue", fill_alpha=0.5)
p.circle(tvals,np.array(solnanalytic)-0.1, size=15, line_color="green",fill_color="green", fill_alpha=0.5)
#p.circle(tvals,np.array(solnanalytic)-np.array(integralvals),size=15, line_color="red", fill_color="red", fill_alpha=0.5)
p.circle(tvals,eintegralvals, size=15, line_color="red", fill_color="red", fill_alpha=0.5)
show(p) # show the results

# Two Body, Earth and Sun

Generate iniitial data for two body problem, Sun and Earth, using reduced mass framework.

In [ ]:
import math
def InitialData():
    random.seed(a=9001)
    
    #initially use an in plane orbit with random starting locations relative to the x axis
    phi=np.array([0,math.pi])
    orbitangle=np.zeros(2)
    #orbitalradius=np.random.uniform(.1,50,2)
    orbitalradius=np.ones(2)
    #start with circular orbits
    #eccentricity=np.random.uniform(0.,.1,2)
    eccentricity=np.zeros(2)
    #magnitude=np.random.uniform(-20,-30,2) #absolute not apparent maginutde
    #magsun=-26.832
    masssun=1.989*10**30
    lsun=3.828*10**26
    massearth=5.9722*10**24
    #luminosity=lsun*10**(0.4*(magnitude-magsun))
    #masses= invertIMF(luminosity,lsun,masssun) #Initial mass function for Main Sequence
    #masses=np.random.uniform(.7,5.) #replace with IMF
    masses=np.array([masssun,massearth])

    return phi,orbitangle,orbitalradius,eccentricity, masses



In [ ]:
import random,numpy as np
initdat=InitialData()

In [ ]:
print(initdat)

Transform cylindrical initial data to x,y,z, velocity, acceleration initial data

In [ ]:
def getxyuv(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    #print(orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=149597870700
    Gconstant=6.408*10**-11
    x0=orbitalradius*np.cos(phi)*metersperAU
    y0=orbitalradius*np.sin(phi)*metersperAU
    z0=np.zeros(2)
    

    
    #start at perihelion for both (eliptical, doesn't generalize to three body)
    #actually start with circular orbit
    ux0=np.zeros(2)*149597870700
    #centrepital force balances gravitational force
    reducedmass=np.zeros(2)
    print(masses)
    for i in np.arange(2):
        j=(i+1)%2 #reverse masses
        reducedmass[i]=masses[i]*masses[j]/np.sum(masses)
    print(reducedmass)
    rphys=np.zeros(len(masses))
    for i in np.arange(2):
        rphys[i]=orbitalradius[i]*metersperAU*reducedmass[i]/masses[i]
    x0=rphys*np.cos(phi)
    y0=rphys*np.sin(phi)
    z0=np.zeros(2)
    print(rphys)
    F=(Gconstant*reducedmass**2/rphys**2)
    print(F)
    #centF=reducedmass*v**2/rphys
    #centF=accel
    v=np.sqrt(Gconstant*reducedmass/rphys)
    print(v)
    ux0=v*np.sin(phi)
    uy0=v*np.cos(phi) #initial data in y only
    #evolve in plane only
    #there is a units problem that needs to be fixed
    #velocity initial conditions are not trivial. 
    uz0=np.zeros(2)
    
    #circular orbit   #a=omega^2 * r #v=omega*r #omega=v/r
    omega=v/rphys
    print(omega)
    omegatrue=np.mean(omega) #shouldaccount for numerical effects
    print(omegatrue)
    a=omegatrue**2*rphys
    print("a",a)
    print(phi)
    ax0=-a*np.cos(phi)
    ay0=-a*np.sin(phi)
    az0=np.zeros(2)
    
    
    return reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [ ]:
xyuva=getxyuv(initdat)
print(xyuva)#In SI units
print(xyuva[1][0])
print(xyuva[0][0]/xyuva[0][1])
print(xyuva)

Time stepping routine using Euler method and Newtonian differential equation for two body system

In [ ]:
def timestep(step,t,dt,reducedmass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vx))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vy))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=axi
    ayii=ayi
    azii=azi
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        #print(xii)
        vxii[m] = vxi[m] + dt*axi[m]
        #print(vxii)
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
    
    
    Gconstant=6.408*10**-11
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                axii[j]+=Gconstant*reducedmass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]+=Gconstant*reducedmass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]+=Gconstant*reducedmass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    return reducedmass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

Run over several time steps and print

In [ ]:
dt=0.01*31556926 #seconds per year
numsteps=5
mass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

Object for storing the orbital differential equation and evolving using implicit RK4 using Newtonian differential equation for two body system

In [ ]:
class OrbitDiffEq:
    def __init__(self,reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0):
        self.reducedmass=reducedmass
        self.xi=x0
        self.yi=y0
        self.zi=z0
        self.vxi=ux0
        self.vyi=uy0
        self.vzi=uz0
        self.axi=ax0
        self.ayi=ay0
        self.azi=az0
    def dxidt(self,t):
        return self.vxi
    def dyidt(self,t):
        return self.vyi
    def dzidt(self,t):
        return self.vzi
    def dvxidt(self,t):
        #return axi[m]
        axii=self.axi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    axii[j]+=Gconstant*self.reducedmass[k]*(self.xi[j]  - self.xi[k])/rreljk**3
        self.axi=axii
        return axii
    def dvyidt(self,t):
        #return axi[m]
        ayii=self.ayi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    ayii[j]+=Gconstant*self.reducedmass[k]*(self.yi[j]  - self.yi[k])/rreljk**3
        self.ayi=ayii
        return ayii
    def dvzidt(self,t):
        #return axi[m]
        azii=self.azi
        rii=np.sqrt(self.xi**2+self.yi**2+self.zi**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((self.xi[j] - self.xi[k])**2+(self.yi[j]-self.yi[k])**2+(self.zi[j]-self.zi[k])**2)**(1./2.)
                    azii[j]+=Gconstant*self.reducedmass[k]*(self.zi[j]  - self.zi[k])/rreljk**3
        self.azi=azii
        return azii
    def updateINTERNAL(self,xii,yii,zii,vxii,vyii,vzii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        return self
    def update(self,xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        self.axi=axii
        self.ayi=ayii
        self.azi=azii
    def print2D(self):
        print(self.xi,self.yi,self.vxi,self.vyi,self.axi,self.ayi)
        return self
    def list2D(self):
        return self.xi,self.yi,self.vxi,self.vyi,self.axi,self.ayi
    def timestepRK4ODE(self,step,t,dt):

    
        h=dt
        #tnew,ynew, intval=RK4(h,t,y,f)
        #m represents choices of mass
        i=step
        
        tnew,intvalx=RK4implicit(h,t,self.dxidt)
        xii = intvalx
        tnew,intvalvx=RK4implicit(h,t,self.dvxidt)
        vxii=intvalvx
        tnew,intvaly=RK4implicit(h,t,self.dyidt)
        yii = intvaly
        tnew,intvalvy=RK4implicit(h,t,self.dvyidt)
        vyii=intvalvy
        tnew,intvalz=RK4implicit(h,t,self.dzidt)
        zii = intvalz
        tnew,intvalvz=RK4implicit(h,t,self.dvzidt)
        vzii=intvalvz
 
        #print(xii)
        self.updateINTERNAL(xii,yii,zii,vxii,vyii,vzii)
        return reducedmass, xii,yii,zii,vxii,vyii,vzii,self.axi,self.ayi,self.azi

Implicit RK4

In [ ]:
def RK4implicit(h,t,f): #not a finite difference so no step in y
    k1= h*f(t)
    k2=h*f(t+h/2.)
    k3=h*f(t+h/2.)
    k4=h*f(t+h)
    return t+h, f(t)+1/6.*(k1+2.*k2+2.*k3+k4)




Initialize iniitial conditions 

In [ ]:
reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0=xyuva
ODE= OrbitDiffEq(reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0)
ODE.print2D()

Extract coordinates

In [ ]:
mass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
print(x,y,vx,vy,ax,ay)
print(xyuva)

Evolve the differential equation and print

In [ ]:
dt=0.001*31556926 #seconds per year
numsteps=5
reducedmass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
ODE.print2D()
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    reducedmass, x,y,z,vx,vy,vz,ax,ay,az=ODE.timestepRK4ODE(i,t,dt)
    ODE.print2D()
    ODE.update(x,y,z,vx,vy,vz,ax,ay,az)



In [ ]:
secperyr=31556926

Evolve the differential equation and store in a pandas data frame for later plotting

In [ ]:
ODE2= OrbitDiffEq(reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0)



In [ ]:
dt=0.01*31556926 #seconds per year
numsteps=5
reducedmass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
datainit=[[0,0.0,x,y,z,vx,vy,vz,ax,ay,az]]
import pandas as pd
columns =['step','time','x' ,'y','z','vx','vy','vz','ax','ay','az']
df = pd.DataFrame(data=datainit, columns=columns)




In [ ]:
display(df)

In [ ]:
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    reducedmass, x,y,z,vx,vy,vz,ax,ay,az=ODE2.timestepRK4ODE(i,t,dt)
    df.loc[i]=i,t,x,y,z,vx,vy,vz,ax,ay,az
    ODE2.update(x,y,z,vx,vy,vz,ax,ay,az)


In [ ]:
display(df)

In [ ]:
xser=df['x']
xser
xser[1][0]

Each pandas column has two entries because there are two bodies, the sun and the earth. These need to be split into columns one and two in a new table for each pandas column in the original table. The following routine automates extraction of these entries for each column. 

In [ ]:
def mergeStarDataIntoDataFrame(col):
    rawdataser=df[col]
    numstars=len(rawdataser[0])
    columns=np.arange(numstars)
    numentries=len(rawdataser)
    index=np.arange(numentries)
    dfnew=pd.DataFrame(columns=columns,index=index)
    for coln in np.arange(numstars):
        datacollist=[]
        for row in np.arange(numentries):
            datacollist.append(rawdataser[row][coln])
        dfnew[coln]=datacollist
    return dfnew

In [ ]:
dfx=mergeStarDataIntoDataFrame('x')

In [ ]:
display(dfx)

In [ ]:
dfy=mergeStarDataIntoDataFrame('y')

In [ ]:
display(dfy)

In [ ]:
dfvx=mergeStarDataIntoDataFrame('vx')

In [ ]:
display(dfvx)

In [ ]:
dfvy=mergeStarDataIntoDataFrame('vy')

In [ ]:
display(dfvy)

In [ ]:
dfax=mergeStarDataIntoDataFrame('ax')

In [ ]:
display(dfax)

In [ ]:
dfay=mergeStarDataIntoDataFrame('ay')

In [ ]:
display(dfay)

In [ ]:
df['time'].tolist()

Plots time versus position of Earth, in reduced mass coordinates

In [ ]:
from bokeh.plotting import figure, output_notebook, show
p=figure(plot_width=300,plot_height=300,x_axis_label="Time (s)",y_axis_label="y displacement (m), Earth")
show(p)
p.circle(x=np.array(df['time'].tolist()),y=np.array(dfy[1].tolist()))
show(p)
output_notebook()